In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhan3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import os
filepath_recipes = os.path.abspath("C:/Users/zhan3/Desktop/RAW_recipes.csv")
filepath_interactions = os.path.abspath("C:/Users/zhan3/Desktop/RAW_interactions.csv")

In [ ]:
df_recipe = pd.read_csv(filepath_recipes)
df_interaction = pd.read_csv(filepath_interactions)
df = pd.merge(df_recipe, df_interaction, how='inner', left_on='id', right_on='recipe_id')

In [ ]:
df_rating = df.groupby('id').mean()['rating']

In [ ]:
df_reviews = df.groupby('id').count()['review']

In [ ]:
df_new = pd.merge(df_recipe, df_rating, how='left', on='id')

In [ ]:
df_new = df_new[['id', 'name', 'tags', 'description', 'ingredients', 'rating']]
df_new.dropna(axis=0, inplace=True)
df_final = df_new.reset_index(drop=True)

In [ ]:
df_final.head()

In [3]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#### Run this cell carefully, it will take 30 minutes.

In [4]:
# for i in range(len(df_final)):
    
#     ingredients = df_final.loc[i]['ingredients']
#     ingredients = re.sub('[^a-zA-Z]', ' ', ingredients)
#     name = df_final.loc[i]['name']
#     name = re.sub('[^a-zA-Z]', ' ', name)
#     combined = name + ingredients
#     combined = combined.split()
#     ps = PorterStemmer()
#     combined = [ps.stem(word) for word in combined if not word in set(stopwords.words('english'))]
#     combined = ' '.join(combined)
#     df_final.at[i, "combined words"] = combined    

In [5]:
# df_final.to_csv('recipe.csv', index=False)

In [6]:
# df_read = pd.read_csv('recipe.csv')
# df_read = pd.merge(df_read, df_reviews, how='left', on='id')
# df_words = df_read[['id', 'combined words', 'rating', 'review']]

In [7]:
# df_words.to_csv('recipe_final.csv')

### NLP training

In [8]:
df_words = pd.read_csv("recipe_final.txt",index_col=[0])
df_words

,id,combined words,rating,review
0,137739,arriba bake winter squash mexican style winter...,5.0,3
1,31490,bit differ breakfast pizza prepar pizza crust ...,3.5,4
2,112140,kitchen chili ground beef yellow onion dice to...,4.0,1
3,59389,alouett potato spreadabl chees garlic herb new...,4.5,2
4,44061,amish tomato ketchup canning tomato juic appl ...,5.0,1
...,...,...,...,...
226652,486161,zydeco soup celeri onion green sweet pepper ga...,5.0,6
226653,493372,zydeco spice mix paprika salt garlic powder on...,5.0,1
226654,308080,zydeco ya ya devil egg hard cook egg mayonnais...,5.0,5
226655,298512,cooki design cooki stick butter eagl brand con...,1.0,1


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Map the word column into an array
text = df_words['combined words'].to_list()

# Calculate the TF-IDF matrix
model = TfidfVectorizer()
matrix = model.fit_transform(text)

In [11]:
def query_preprocessing(query):
    
    query = re.sub('[^a-zA-Z]', ' ', query)
    query = query.split()
    ps = PorterStemmer()
    query = [ps.stem(word) for word in query if not word in set(stopwords.words('english'))]
    query = ' '.join(query)
    
    return query

In [18]:
def get_recipe(query):
    
    query = query_preprocessing(query)
    query_matrix = model.transform([query])
    similarity = cosine_similarity(matrix, query_matrix)
    
    similarity.flatten()
    top10 = np.argsort(similarity.flatten())[::-1][:100]
    top10sorted = np.sort(similarity.flatten())[::-1][:100]
    length = len(top10)
    results = []
    
    for i in range(length):
        j = top10[i]
        name = df_words.loc[j]['combined words']
        rating = df_words.loc[j]['rating']
        review = df_words.loc[j]['review']
        tmp = top10sorted[i] * rating * review
        results.append(tmp)
        
    resultssorted = np.argsort(results)
    bestindex = top10[resultssorted[-1]]
    print(df_words.loc[bestindex])
    
    return name

In [19]:
print(get_recipe('arugula salad'))

id                                                           321928
combined words    parmesan crust chicken arugula salad dijon mus...
rating                                                      4.58824
review                                                           17
Name: 149747, dtype: object
dri fig goat chees arugula salad walnut rosemari honey sherri wine vinegar water dijon mustard salt pepper extra virgin oliv oil arugula dri fig goat chees
